<a href="https://colab.research.google.com/github/Vincent-ondeng/Final_year_research_project/blob/main/Notebook/combined_webmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the neccessary libraries and dependencies
!pip install folium
!pip install geopandas
!pip install pandas
!pip install numpy

In [20]:
#Import Libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium.features import GeoJsonTooltip
from folium.plugins import MarkerCluster

In [21]:
# Read the GeoJSON file using Geopandas
geojson = gpd.read_file("/content/wards.geojson")
#geojson=geojson[['wardcode','geometry']] #only select 'wardcode' (Ward_Code) and 'geometry' columns

geojson2 = gpd.read_file("/content/subcounty.geojson")
geojson2=geojson2[['FID_','geometry']] #only select 'FID' and 'geometry' columns


In [22]:
df = pd.read_csv('/content/facilities_coordinates.csv')
# df.head()

In [23]:
#Read the covid-19 data using pandas
malaria_df=pd.read_csv("/content/wards.csv")
subcounty_df=pd.read_csv("/content/subcounty.csv")

In [ ]:
geojson.head()
geojson2.head()
malaria_df.head()
subcounty_df.head()

In [25]:
#Join the geojson file with malaria_df
df_final = geojson.merge(malaria_df, left_on="wardcode", right_on="wardcode", how="outer")

In [26]:
#Join the geojson file with malaria_df
df_final2 = geojson2.merge(subcounty_df, left_on="FID_", right_on="FID_", how="outer")

In [ ]:
df_final.head()
df_final2.head()

In [28]:
# Create a map instance
m = folium.Map(location=[-0.15386345206107715, 34.86414568574279],
               zoom_start=10,
               control_scale=True,
               prefer_canvas=True)

In [ ]:
choropleth=folium.Choropleth(
    geo_data=geojson,
    name="County Assembly ward malaria distribution",
    data=malaria_df,
    columns=['wardcode', 'Total_co'],
    key_on='feature.properties.wardcode',
    nan_fill_color="White", #Use white color if there is no data available for the county
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="County Assembly Wards Legend",
    line_color='black',
    highlight=True,
).add_to(m)
#Add Customized Tooltips to the map
folium.features.GeoJson(
                    data=df_final,
                    #name='Total Confirmed cases of Malaria in Kisumu',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['wardcode',
                                'ward',
                                'Confirme',
                                'Confirm1',
                                'Malaria_',
                                'Total_co'
                               ],
                        aliases=["WardCode:",
                                 "Ward Name:",
                                 "Confirmed Cases in Children under agve 5:",
                                 "Confirmed Cases in persons greater than age 5:",
                                 "Confirmed Cases in expectant women:",
                                 "Total Confirmed Malaria Cases:"
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(choropleth)

In [ ]:
choropleth1=folium.Choropleth(
    geo_data=geojson2,
    name="Malaria Distribution by Sub-county",
    data=subcounty_df,
    columns=['FID_', 'Total Confirmed Cases in All age groups'],
    key_on='feature.properties.FID_',
    nan_fill_color="White", #Use white color if there is no data available for the county
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Sub-county Distribution Legend",
    line_color='black',
    highlight=True,
).add_to(m)
#Add Customized Tooltips to the map
folium.features.GeoJson(
                    data=df_final2,
                    name='Total Confirmed cases of Malaria in Kisumu',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['Sub-county Name',
                                'Confirmed Malaria (only Positive cases) <5 yrs',
                                'Confirmed Malaria (only Positive cases) >5 yrs',
                                'Malaria in pregnancy >5 yrs',
                                'Total Confirmed Cases in All age groups',
                                'Malaria Incidence per 1000'
                               ],
                        aliases=["Sub-county Name:",
                                 "Confirmed Malaria (only Positive cases) <5 yrs:",
                                 "Confirmed Malaria (only Positive cases) >5 yrs:",
                                 "Malaria in pregnancy >5 yrs:",
                                 "Total Confirmed Cases in All age groups:",
                                 "Malaria Incidence per 1000:"
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(choropleth1)

In [31]:
location = df['Latitude'].mean(), df['Longitude'].mean()

In [32]:
markerCluster = MarkerCluster().add_to(m)

In [33]:
def popup_html(row):
  i = row
  facility_name=df['Facility'].iloc[i]
  malaria_in_children=df['Confirmed Malaria (only Positive cases) <5 yrs'].iloc[i]
  greater_than_5=df['Confirmed Malaria (only Positive cases) >5 yrs'].iloc[i]
  pregnant_women=df['Malaria in pregnancy >5 yrs'].iloc[i]
  total_cases=df['Confirmed Malaria cases in all age groups'].iloc[i]

  html = """
<!DOCTYPE html>
<html>
<tbody>
<tr>
<td style="text-align: left;"><span><b>Facility Name:</b></span></td>
<td style="text-align:right;">"""+facility_name + """</td>
</tr><br>
<tr>
<td style="text-align: left;"><span><b>Confirmed Malaria (only Positive cases) <5 yrs:</b></span></td>
<td style="padding: 2px; padding-right:8px;">{}</td>""".format(malaria_in_children) + """
</tr><br>
<tr>
<td style="text-align: left;"><span><b>Confirmed Malaria (only Positive cases) >5 yrs:<b></span></td>
<td style="padding: 2px; padding-right:8px;">{}</td>""".format(greater_than_5) + """
</tr><br>
<tr>
<td style="text-align: left;"><span><b>Malaria in pregnancy >5 yrs:</b></span></td>
<td style="padding: 2px; padding-right:8px;">{}</td>""".format(pregnant_women) + """</td>
</tr><br>
<tr>
<td style="text-align: left;"><span><b>Confirmed Malaria cases in all age groups:</b></span></td>
<td style="padding: 2px; padding-right:8px;">{}</td>""".format(total_cases) + """
</tr>
</tbody>
</html>
"""
  return html

In [34]:
for i in range(0,len(df)):

  html = popup_html(i)
  popup = folium.Popup(folium.Html(html, script=True), max_width=500)
  Health_facility=folium.CircleMarker([df['Latitude'].iloc[i], df['Longitude'].iloc[i]], popup=popup).add_to(markerCluster)

In [ ]:
folium.TileLayer('OpenStreetMap',overlay=True,name="Street View").add_to(m)
folium.TileLayer('Stamen Terrain',overlay=True,name="Terrain").add_to(m)

In [36]:
folium.LayerControl(collapsed=True).add_to(m)
m

AssertionError: ignored